# Versão original

In [1]:
import pandas as pd
# Carregamento e manipulação de dados mais robusto
import seaborn as sns
import plotly.express as px
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# Visualização dos dados
import matplotlib.pyplot as plt
# Divisão dos dados de treino e de teste
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
# Metricas de avaliação
from sklearn import metrics
import joblib
# Random Forest
from sklearn.ensemble import RandomForestClassifier
# K-NM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [2]:
url = "https://info.dengue.mat.br/api/alertcity"
geocode = 2304400
disease = "dengue"
format = "csv"
ew_start = 1
ew_end = 53
ey_start = 2015
ey_end = 2024

params =(
    "&disease="
    + f"{disease}"
    + "&geocode="
    + f"{geocode}"
    + "&disease="
    + f"{disease}"
    + "&format="
    + f"{format}"
    + "&ew_start="
    + f"{ew_start}"
    + "&ew_end="
    + f"{ew_end}"
    + "&ey_start="
    + f"{ey_start}"
    + "&ey_end="
    + f"{ey_end}"
)

url_resp = "?".join([url, params])
url_resp

'https://info.dengue.mat.br/api/alertcity?&disease=dengue&geocode=2304400&disease=dengue&format=csv&ew_start=1&ew_end=53&ey_start=2015&ey_end=2024'

In [3]:
dados = pd.read_csv(url_resp, index_col='SE')

In [4]:
df = pd.read_csv('/content/sample_data/FORTALEZA_2015_2024.CSV', encoding='latin1' ,sep=";", decimal=',')
#df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] = df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].fillna(0)
df

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)"
0,01/01/2021,0000 UTC,0.0,1009.5,1009.5,1008.9,NaN,27.6,22.7,27.7,27.5,22.9,22.6,75.0,74.0,75.0,87.0,8.3,3.0
1,01/01/2021,0100 UTC,0.0,1009.8,1009.8,1009.5,NaN,27.5,22.4,27.6,27.4,22.7,22.4,75.0,74.0,74.0,90.0,8.4,3.6
2,01/01/2021,0200 UTC,0.0,1010.0,1010.2,1009.8,NaN,27.4,22.6,27.5,27.4,22.7,22.4,75.0,74.0,75.0,98.0,8.4,2.8
3,01/01/2021,0300 UTC,0.0,1009.8,1010.0,1009.8,NaN,27.4,22.6,27.4,27.3,22.9,22.6,76.0,75.0,75.0,99.0,6.9,3.6
4,01/01/2021,0400 UTC,0.0,1009.2,1009.8,1009.2,NaN,27.0,22.5,27.5,27.0,22.7,22.5,76.0,75.0,76.0,102.0,7.2,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22239,16/07/2023,1500 UTC,0.0,1011.8,1012.7,1011.8,1790.2,29.6,21.8,30.5,28.5,21.9,20.5,65.0,57.0,63.0,112.0,11.1,4.1
22240,16/07/2023,1600 UTC,0.0,1010.8,1011.9,1010.8,1904.2,30.0,21.3,30.3,28.8,22.2,21.0,65.0,60.0,60.0,114.0,9.0,4.3
22241,16/07/2023,1700 UTC,0.0,1009.9,1010.8,1009.9,2113.3,29.8,21.4,30.6,29.7,21.9,20.8,61.0,58.0,61.0,100.0,9.0,4.3
22242,16/07/2023,1800 UTC,0.0,1009.5,1010.0,1009.5,1776.5,29.6,21.3,30.2,29.6,22.3,21.1,64.0,59.0,61.0,95.0,8.6,3.5


In [5]:
# prompt: alterar o valor -9999 para np.nan

import numpy as np

# Substituindo -9999 por np.nan em todo o DataFrame
df = df.replace(-9999, np.nan)

In [6]:
# Formatar Data e criamos uma ramificação só pegando os valores Mensal

df['Data'] = pd.to_datetime(df['Data'],format='%d/%m/%Y')
df.set_index('Data', inplace=True)

In [7]:
condicoes={'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)':"sum",
'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)':"mean",
'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)':"mean",
'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)':"mean",
'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)':"mean",
'TEMPERATURA DO PONTO DE ORVALHO (°C)':"mean",
'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)':"mean",
'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)':"mean",
'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)':"mean",
}
# condicoes={'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)':"sum",
# 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)':"mean",
# 'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)':"mean",
# 'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)':"mean",
# 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)':"mean",
# 'TEMPERATURA DO PONTO DE ORVALHO (°C)':"mean",
# 'Temperatura':"mean",
# 'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)':"mean",
# 'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)':"mean",
# 'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)':"mean",
# 'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)':"mean",
# 'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)':"mean",
# 'RADIACAO GLOBAL (Kj/m²)':"mean",
#            'UMIDADE RELATIVA DO AR, HORARIA (%)':"mean",
#            'VENTO, DIREÇÃO HORARIA (gr) (° (gr))':"mean",
#            'VENTO, RAJADA MAXIMA (m/s)':"mean",
#            'VENTO, VELOCIDADE HORARIA (m/s)':"mean"}

In [8]:
columns_to_drop_mensal_sem_vento = ['Hora UTC', 'RADIACAO GLOBAL (Kj/m²)',
                   'UMIDADE RELATIVA DO AR, HORARIA (%)',
                   'VENTO, DIREÇÃO HORARIA (gr) (° (gr))',
                   'VENTO, RAJADA MAXIMA (m/s)', 'VENTO, VELOCIDADE HORARIA (m/s)']

df1 = df.copy()



In [9]:
# df1.drop(columns=columns_to_drop_mensal_sem_vento, inplace=True) AQUI
df1 = df1.resample('W').agg(condicoes)

In [10]:
for coluna in df1.columns:
  df1[coluna]=df1[coluna].round(2)

In [11]:
df1['data_iniSE']=df1.index

In [12]:
df1['data_iniSE']=df1['data_iniSE'].astype(str)

In [13]:
#dados['data_iniSE']=pd.to_datetime(dados['data_iniSE'],format='%Y-%m-%d')

In [14]:
dados.columns

Index(['data_iniSE', 'casos_est', 'casos_est_min', 'casos_est_max', 'casos',
       'p_rt1', 'p_inc100k', 'Localidade_id', 'nivel', 'id', 'versao_modelo',
       'tweet', 'Rt', 'pop', 'tempmin', 'umidmax', 'receptivo', 'transmissao',
       'nivel_inc', 'umidmed', 'umidmin', 'tempmed', 'tempmax', 'casprov',
       'casprov_est', 'casprov_est_min', 'casprov_est_max', 'casconf',
       'notif_accum_year'],
      dtype='object')

In [15]:
# cruzamento_dados=df1.merge(dados[['data_iniSE','nivel', 'casos_est','umidmax', 'tempmax', 'Rt', 'p_rt1', 'nivel_inc', 'transmissao',]],on='data_iniSE',how='left')

In [16]:
cruzamento_dados=df1.merge(dados[['data_iniSE','nivel', 'casos','umidmax', 'tempmax',]],on='data_iniSE',how='left')

In [17]:
cruzamento_dados.dropna(inplace=True)

In [18]:
cruzamento_dados['nivel'].value_counts()

,count
nivel,
1,70
3,23
4,21
2,19


In [19]:
# cruzamento_dados.drop(columns='data_iniSE', inplace=True)

In [20]:
dados

,data_iniSE,casos_est,casos_est_min,casos_est_max,casos,p_rt1,p_inc100k,Localidade_id,nivel,id,...,umidmed,umidmin,tempmed,tempmax,casprov,casprov_est,casprov_est_min,casprov_est_max,casconf,notif_accum_year
SE,,,,,,,,,,,,,,,,,,,,,
202452,2024-12-22,42.0,42,42,42,0.000558,1.617776,0,1,230440020245220248,...,75.053529,57.029057,27.772543,31.019929,5.0,NaN,NaN,NaN,NaN,260570
202451,2024-12-15,65.0,65,65,65,0.028380,2.503701,0,1,230440020245120248,...,78.271629,61.219586,27.464300,30.684629,27.0,NaN,NaN,NaN,NaN,260570
202450,2024-12-08,82.0,82,82,82,0.079278,3.158515,0,1,230440020245020248,...,75.892571,59.095457,27.272271,30.281200,15.0,NaN,NaN,NaN,NaN,260570
202449,2024-12-01,85.0,85,85,85,0.039801,3.274070,0,1,230440020244920248,...,76.211986,58.876143,27.133600,30.184143,23.0,NaN,NaN,NaN,NaN,260570
202448,2024-11-24,99.0,99,99,99,0.340759,3.813329,0,1,230440020244820248,...,70.823429,49.955500,27.747600,31.314629,34.0,NaN,NaN,NaN,NaN,260570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201505,2015-02-01,311.0,311,311,311,0.999998,11.575918,0,3,230440020150519589,...,67.709184,48.714286,27.913265,31.000000,154.0,NaN,NaN,NaN,NaN,260570
201504,2015-01-25,221.0,221,221,221,0.989142,8.225974,0,3,230440020150419589,...,63.959183,44.571429,27.545918,30.285714,106.0,NaN,NaN,NaN,NaN,260570
201503,2015-01-18,207.0,207,207,207,0.999989,7.704871,0,3,230440020150319589,...,66.280613,50.142857,27.515306,30.000000,79.0,NaN,NaN,NaN,NaN,260570


In [21]:
cruzamento_dados

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),data_iniSE,nivel,casos,umidmax,tempmax
0,1.0,1008.17,1008.39,1007.98,28.49,21.64,29.03,28.05,22.05,21.24,69.57,65.18,2021-01-03,1,123,84.629829,31.000000
1,16.4,1008.11,1008.35,1007.87,28.35,21.71,28.83,27.94,22.13,21.31,70.14,65.67,2021-01-10,1,116,82.266077,31.000000
2,3.6,1007.68,1007.91,1007.46,28.19,21.24,28.72,27.71,21.68,20.76,69.16,64.30,2021-01-17,1,130,89.567574,29.857143
3,47.0,1008.19,1008.45,1007.93,27.02,22.06,27.45,26.59,22.41,21.66,76.99,73.04,2021-01-24,1,127,88.094185,30.000000
4,53.8,1007.70,1007.95,1007.47,27.61,22.29,28.05,27.23,22.73,21.89,75.56,71.46,2021-01-31,1,164,83.702743,30.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,24.2,1010.75,1011.00,1010.50,26.95,22.07,27.46,26.53,22.57,21.65,77.14,71.73,2023-06-18,1,309,83.757649,27.052632
129,3.0,1010.99,1011.21,1010.78,27.07,21.34,27.53,26.66,21.81,20.94,74.12,69.51,2023-06-25,1,252,83.413169,26.736842
130,12.6,1010.97,1011.20,1010.74,26.70,21.08,27.22,26.24,21.60,20.66,74.66,69.55,2023-07-02,1,313,86.280823,27.000000
131,13.0,1010.97,1011.18,1010.75,26.93,21.46,27.40,26.48,21.99,21.05,75.08,69.74,2023-07-09,2,302,87.070000,29.393057


In [22]:
cruzamento_dados['data_iniSE'] = pd.to_datetime(cruzamento_dados['data_iniSE'],format='%Y-%m-%d')

In [23]:
cruzamento_dados['dia'] = cruzamento_dados['data_iniSE'].dt.day
cruzamento_dados['mes'] = cruzamento_dados['data_iniSE'].dt.month
cruzamento_dados

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),data_iniSE,nivel,casos,umidmax,tempmax,dia,mes
0,1.0,1008.17,1008.39,1007.98,28.49,21.64,29.03,28.05,22.05,21.24,69.57,65.18,2021-01-03,1,123,84.629829,31.000000,3,1
1,16.4,1008.11,1008.35,1007.87,28.35,21.71,28.83,27.94,22.13,21.31,70.14,65.67,2021-01-10,1,116,82.266077,31.000000,10,1
2,3.6,1007.68,1007.91,1007.46,28.19,21.24,28.72,27.71,21.68,20.76,69.16,64.30,2021-01-17,1,130,89.567574,29.857143,17,1
3,47.0,1008.19,1008.45,1007.93,27.02,22.06,27.45,26.59,22.41,21.66,76.99,73.04,2021-01-24,1,127,88.094185,30.000000,24,1
4,53.8,1007.70,1007.95,1007.47,27.61,22.29,28.05,27.23,22.73,21.89,75.56,71.46,2021-01-31,1,164,83.702743,30.285714,31,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,24.2,1010.75,1011.00,1010.50,26.95,22.07,27.46,26.53,22.57,21.65,77.14,71.73,2023-06-18,1,309,83.757649,27.052632,18,6
129,3.0,1010.99,1011.21,1010.78,27.07,21.34,27.53,26.66,21.81,20.94,74.12,69.51,2023-06-25,1,252,83.413169,26.736842,25,6
130,12.6,1010.97,1011.20,1010.74,26.70,21.08,27.22,26.24,21.60,20.66,74.66,69.55,2023-07-02,1,313,86.280823,27.000000,2,7
131,13.0,1010.97,1011.18,1010.75,26.93,21.46,27.40,26.48,21.99,21.05,75.08,69.74,2023-07-09,2,302,87.070000,29.393057,9,7


In [24]:
# cruzamento_dados['nivel_lag1'] = cruzamento_dados['nivel'].shift(1)
# cruzamento_dados['nivel_lag1'] = cruzamento_dados['nivel_lag1'].fillna(cruzamento_dados['nivel'][0])
# cruzamento_dados['Rt_lag1'] = cruzamento_dados['Rt'].shift(1)
# cruzamento_dados['Rt_lag1'] = cruzamento_dados['Rt_lag1'].fillna(cruzamento_dados['Rt'][0])
# cruzamento_dados['p_rt1_lag1'] = cruzamento_dados['p_rt1'].shift(1)
# cruzamento_dados['p_rt1_lag1'] = cruzamento_dados['p_rt1_lag1'].fillna(cruzamento_dados['p_rt1'][0])
# cruzamento_dados['nivel_inc_lag1'] = cruzamento_dados['nivel_inc'].shift(1)
# cruzamento_dados['nivel_inc_lag1'] = cruzamento_dados['p_rt1_lag1'].fillna(cruzamento_dados['nivel_inc'][0])

In [25]:
cruzamento_dados.columns

Index(['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
       'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
       'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
       'TEMPERATURA DO PONTO DE ORVALHO (°C)',
       'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)', 'data_iniSE', 'nivel',
       'casos', 'umidmax', 'tempmax', 'dia', 'mes'],
      dtype='object')

In [26]:
# COMEÇAR OS TESTES (NAO SEI O QUE ESTOU FAZENDO)
# X => Atributos de entrada usados para prever a minha saida (y)
X = cruzamento_dados[[
   'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
    # 'Rt_lag1',
    # 'p_rt1_lag1',
    #'nivel_inc_lag1',
    # 'transmissao',
   'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
 #  'nivel_lag1',
      'dia',
    'mes'
]]
# X = cruzamento_dados[[
#     'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
#        'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
#        'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
#        'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
#        'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
#        'TEMPERATURA DO PONTO DE ORVALHO (°C)', 'Temperatura',
#        'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
#        'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
#        'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
#        'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
#        'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
#     'RADIACAO GLOBAL (Kj/m²)',
#        'UMIDADE RELATIVA DO AR, HORARIA (%)',
#        'VENTO, DIREÇÃO HORARIA (gr) (° (gr))', 'VENTO, RAJADA MAXIMA (m/s)',
#        'VENTO, VELOCIDADE HORARIA (m/s)'
# ]]

y = cruzamento_dados["nivel"]

In [27]:
# CRIAR LAGS DAS VARIÁVEIS
met_vars = [
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
    'mes',
    'dia'
]
for var in met_vars:
    for lag in [1,2,3,4]:
        cruzamento_dados[f'{var}_lag{lag}'] = cruzamento_dados[var].shift(lag)

In [28]:
#LIMPAR AS LINHAS COM NAN (os primeiros lags)
cruzamento_dados.dropna(inplace=True)

In [29]:
# Função para checar se a variável é met_var ou seu lag
def is_met_var(var_name):
    for met in met_vars:
        if var_name == met or var_name.startswith(met + "_lag"):
            return True
    return False

# Selecionar só met_vars e seus lags
features = [c for c in cruzamento_dados.columns if is_met_var(c)]

# Agora X só terá as variáveis desejadas
X = cruzamento_dados[features]
y = cruzamento_dados['casos']

In [30]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# 5) SPLIT TEMPORAL (70% TREINO, 30% TESTE)
n = len(cruzamento_dados)
n_train = int(0.7 * n)

X_train = X.iloc[:n_train]
X_test  = X.iloc[n_train:]
y_train = y.iloc[:n_train]
y_test  = y.iloc[n_train:]

In [31]:
#scaler = MinMaxScaler()
#scaler.fit(data)

In [32]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [33]:
resultado = decision_tree.predict(X_test)

In [34]:
metrics.recall_score(y_test, resultado, average="micro")

0.0

In [35]:
print(classification_report(y_test, resultado))

              precision    recall  f1-score   support

          19       0.00      0.00      0.00       0.0
          98       0.00      0.00      0.00       1.0
         102       0.00      0.00      0.00       1.0
         115       0.00      0.00      0.00       0.0
         118       0.00      0.00      0.00       1.0
         140       0.00      0.00      0.00       1.0
         159       0.00      0.00      0.00       1.0
         160       0.00      0.00      0.00       1.0
         164       0.00      0.00      0.00       0.0
         169       0.00      0.00      0.00       0.0
         173       0.00      0.00      0.00       1.0
         176       0.00      0.00      0.00       1.0
         187       0.00      0.00      0.00       1.0
         192       0.00      0.00      0.00       0.0
         200       0.00      0.00      0.00       1.0
         210       0.00      0.00      0.00       0.0
         236       0.00      0.00      0.00       0.0
         244       0.00    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [36]:
y_test

,casos
94,160
95,159
96,200
97,254
98,176
99,173
100,118
101,102
102,98
103,140


In [37]:
resultado

array([ 336,  236,  297, 1697,   19,  192, 1368,   19,   19,   19,  210,
        768,  169,  115,  164,  187,  298,  322,  244,  490,  780,  251,
        251, 1803, 1320, 1501, 1776, 1944, 2269, 1414, 1414, 1414, 2289,
       1414, 1370, 1370,  904,  747, 1370])

In [38]:
for i in range(1, 102,2):
  knm = KNeighborsClassifier(n_neighbors=i, metric='euclidean')
  knm.fit(X_train, y_train)
  resultado_knm = knm.predict(X_test)
  print(i,metrics.recall_score(y_test, resultado_knm, average="micro"))
  print(classification_report(y_test, resultado_knm))

1 0.0
              precision    recall  f1-score   support

          19       0.00      0.00      0.00       0.0
          98       0.00      0.00      0.00       1.0
         102       0.00      0.00      0.00       1.0
         115       0.00      0.00      0.00       0.0
         118       0.00      0.00      0.00       1.0
         138       0.00      0.00      0.00       0.0
         140       0.00      0.00      0.00       1.0
         159       0.00      0.00      0.00       1.0
         160       0.00      0.00      0.00       1.0
         169       0.00      0.00      0.00       0.0
         173       0.00      0.00      0.00       1.0
         176       0.00      0.00      0.00       1.0
         187       0.00      0.00      0.00       1.0
         192       0.00      0.00      0.00       0.0
         200       0.00      0.00      0.00       1.0
         204       0.00      0.00      0.00       0.0
         210       0.00      0.00      0.00       0.0
         221       0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

          19       0.00      0.00      0.00       0.0
          98       0.00      0.00      0.00       1.0
         102       0.00      0.00      0.00       1.0
         115       0.00      0.00      0.00       0.0
         118       0.00      0.00      0.00       1.0
         133       0.00      0.00      0.00       0.0
         138       0.00      0.00      0.00       0.0
         140       0.00      0.00      0.00       1.0
         159       0.00      0.00      0.00       1.0
         160       0.00      0.00      0.00       1.0
         164       0.00      0.00      0.00       0.0
         169       0.00      0.00      0.00       0.0
         173       0.00      0.00      0.00       1.0
         176       0.00      0.00      0.00       1.0
         187       0.00      0.00      0.00       1.0
         191       0.00      0.00      0.00       0.0
         192       0.00      0.00      0.00       0.0
         195       0.00    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

          19       0.00      0.00      0.00       0.0
          98       0.00      0.00      0.00       1.0
         102       0.00      0.00      0.00       1.0
         115       0.00      0.00      0.00       0.0
         118       0.00      0.00      0.00       1.0
         133       0.00      0.00      0.00       0.0
         138       0.00      0.00      0.00       0.0
         140       0.00      0.00      0.00       1.0
         159       0.00      0.00      0.00       1.0
         160       0.00      0.00      0.00       1.0
         173       0.00      0.00      0.00       1.0
         176       0.00      0.00      0.00       1.0
         187       0.00      0.00      0.00       1.0
         191       0.00      0.00      0.00       0.0
         192       0.00      0.00      0.00       0.0
         195       0.00      0.00      0.00       0.0
         200       0.00      0.00      0.00       1.0
         244       0.00    

...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

          19       0.00      0.00      0.00       0.0
          98       0.00      0.00      0.00       1.0
         102       0.00      0.00      0.00       1.0
         118       0.00      0.00      0.00       1.0
         140       0.00      0.00      0.00       1.0
         159       0.00      0.00      0.00       1.0
         160       0.00      0.00      0.00       1.0
         173       0.00      0.00      0.00       1.0
         176       0.00      0.00      0.00       1.0
         187       0.00      0.00      0.00       1.0
         191       0.00      0.00      0.00       0.0
         200       0.00      0.00      0.00       1.0
         244       0.00      0.00      0.00       1.0
         252       0.00      0.00      0.00       1.0
         254       0.00      0.00      0.00       1.0
         255       0.00      0.00      0.00       1.0
         263       0.00      0.00      0.00       1.0
         265       0.00    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

          98       0.00      0.00      0.00       1.0
         102       0.00      0.00      0.00       1.0
         118       0.00      0.00      0.00       1.0
         140       0.00      0.00      0.00       1.0
         159       0.00      0.00      0.00       1.0
         160       0.00      0.00      0.00       1.0
         173       0.00      0.00      0.00       1.0
         176       0.00      0.00      0.00       1.0
         187       0.00      0.00      0.00       1.0
         191       0.00      0.00      0.00       0.0
         200       0.00      0.00      0.00       1.0
         244       0.00      0.00      0.00       1.0
         252       0.00      0.00      0.00       1.0
         254       0.00      0.00      0.00       1.0
         255       0.00      0.00      0.00       1.0
         263       0.00      0.00      0.00       1.0
         265       0.00      0.00      0.00       1.0
         270       0.00    

ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 91, n_samples_fit = 90, n_samples = 39

In [ ]:
valores=pd.DataFrame(columns=['indice','porcentagem'])
# for i in range(1, 100):
random_forest = RandomForestClassifier(n_estimators=18)
random_forest.fit(X_train, y_train)
resultado = random_forest.predict(X_test)

resultado_porcentagem=metrics.recall_score(y_test, resultado, average="micro")
print(classification_report(y_test, resultado))

valores.loc[len(valores)] = [18, resultado_porcentagem]
print(18,metrics.recall_score(y_test, resultado, average="micro"))

In [ ]:
display(valores)

In [ ]:
# prompt: pegar o valor da coluna indice, onde tiver o maior valor da coluna porcentagem

# Find the index with the maximum 'porcentagem' value
max_porcentagem_index = valores['porcentagem'].idxmax()

# Get the corresponding 'indice' value
indice_at_max_porcentagem = valores.loc[max_porcentagem_index, 'indice']

print(f"The 'indice' value corresponding to the maximum 'porcentagem' is: {indice_at_max_porcentagem}, valor {valores['porcentagem'].max()}")


In [ ]:
importances = random_forest.feature_importances_
for name, score in zip(X.columns, importances):
    print(f"{name}: {score:.2f}")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Já temos importances e X.columns:
importances = random_forest.feature_importances_

# Criar DataFrame
df = pd.DataFrame({'Variável': X.columns, 'Importância': importances})

# Extrair nome base
df['Variável_base'] = df['Variável'].str.extract(r'(^[^\(_]+)')

# Agrupar e somar importâncias
df_grouped = df.groupby('Variável_base')['Importância'].sum().reset_index()

# Ordenar
df_grouped = df_grouped.sort_values(by='Importância', ascending=True)

# Plotar gráfico
plt.figure(figsize=(12, 8))
plt.barh(df_grouped['Variável_base'], df_grouped['Importância'], color='forestgreen')
plt.title('Importância das Variáveis (Random Forest) - Agrupada por 4 semanas', fontsize=24, weight='bold')
plt.xlabel('Importância', fontsize=18)
plt.ylabel('Variável', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.gca().set_facecolor('white')
plt.tight_layout()
plt.savefig("importancia_random_forest_banner.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Gera a matriz de confusão
cm = confusion_matrix(y_test, resultado, labels=decision_tree.classes_)

# Cria o gráfico com os nomes das classes
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=decision_tree.classes_)

fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax, cmap='Blues', colorbar=False)

plt.title("Matriz de Confusão - Classificação de Nível da Dengue")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.tight_layout()
plt.show()

✅ Esse gráfico vai mostrar:
| Real \ Previsto | Baixo | Médio | Alto |
| --------------- | ----- | ----- | ---- |
| **Baixo**       | 100   | 3     | 0    |
| **Médio**       | 8     | 70    | 5    |
| **Alto**        | 0     | 4     | 30   |


✅ Ou seja:

Quantos níveis Baixo foram corretamente previstos como Baixo

Quantos Médio o modelo confundiu com Alto, etc.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Gera a matriz de confusão
cm = confusion_matrix(y_test, resultado, labels=decision_tree.classes_)

# Cria o gráfico com estilo profissional
fig, ax = plt.subplots(figsize=(10, 8))  # Tamanho maior para banner

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=decision_tree.classes_)

disp.plot(ax=ax, cmap='Blues', colorbar=False)

# Ajustes de estilo
plt.title("Matriz de Confusão\nClassificação do Nível da Dengue - Fortaleza (2015–2024)",
          fontsize=20, weight='bold', pad=20)
plt.xlabel("Previsão do Modelo", fontsize=16)
plt.ylabel("Valor Real", fontsize=16)

# Ajuste nos ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Fundo branco
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

plt.tight_layout()
plt.show()


In [ ]:
plt.savefig("matriz_confusao_banner.png", dpi=300)


# Versão atualizada

In [ ]:
import pandas as pd
import seaborn as sns
import plotly.express as px
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import numpy as np

## Obter dados através da api da Info Dengue
Documentação: [Tutorial da API](https://info.dengue.mat.br/services/tutorial/Python)


In [ ]:
url = "https://info.dengue.mat.br/api/alertcity"
geocode = 2304400
disease = "dengue"
format = "csv"
ew_start = 1
ew_end = 53
ey_start = 2015
ey_end = 2024

params =(
    "&disease="
    + f"{disease}"
    + "&geocode="
    + f"{geocode}"
    + "&disease="
    + f"{disease}"
    + "&format="
    + f"{format}"
    + "&ew_start="
    + f"{ew_start}"
    + "&ew_end="
    + f"{ew_end}"
    + "&ey_start="
    + f"{ey_start}"
    + "&ey_end="
    + f"{ey_end}"
)

url_resp = "?".join([url, params])
url_resp

## Ler os dados metereologicos pelo site INMET (Instituto nacional de metereologia)
[Link](https://portal.inmet.gov.br/dadoshistoricos)

In [ ]:
df = pd.read_csv('/content/data/FORTALEZA_2015_2024.CSV', encoding='latin1' ,sep=";", decimal=',')

### Limpeza e tratamento de dados

#### Substituindo -9999 por np.nan em todo o DataFrame



In [ ]:
df = df.replace(-9999, np.nan)

#### Formatar Data e criamos uma ramificação só pegando os valores Mensal


In [ ]:
df['Data'] = pd.to_datetime(df['Data'],format='%d/%m/%Y')
df.set_index('Data', inplace=True)

#### Condições

In [ ]:
condicoes={'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)':"sum",
'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)':"mean",
'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)':"mean",
'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)':"mean",
'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)':"mean",
'TEMPERATURA DO PONTO DE ORVALHO (°C)':"mean",
'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)':"mean",
'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)':"mean",
'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)':"mean",
}


#### Agrupar o dataframe com suas condições

#### Colunas para remover

In [ ]:
columns_to_drop_mensal_sem_vento = ['Hora UTC', 'RADIACAO GLOBAL (Kj/m²)',
                   'UMIDADE RELATIVA DO AR, HORARIA (%)',
                   'VENTO, DIREÇÃO HORARIA (gr) (° (gr))',
                   'VENTO, RAJADA MAXIMA (m/s)', 'VENTO, VELOCIDADE HORARIA (m/s)']
df1 = df.copy()

In [ ]:
df1 = df1.resample('W').agg(condicoes)

#### Arredondamento

In [ ]:
for coluna in df1.columns:
  df1[coluna]=df1[coluna].round(2)

In [ ]:
df1['data_iniSE']=df1.index

In [ ]:
df1['data_iniSE']=df1['data_iniSE'].astype(str)

## Obter informações sobre o DataFrame

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
# CRIAR LAGS DAS VARIÁVEIS
met_vars = [
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
    'mes',
    'dia'
]
for var in met_vars:
    for lag in [1,2,3,4]:
        cruzamento_dados[f'{var}_lag{lag}'] = cruzamento_dados[var].shift(lag)

In [ ]:
#LIMPAR AS LINHAS COM NAN (os primeiros lags)
cruzamento_dados.dropna(inplace=True)

In [ ]:
# Função para checar se a variável é met_var ou seu lag
def is_met_var(var_name):
    for met in met_vars:
        if var_name == met or var_name.startswith(met + "_lag"):
            return True
    return False

# Selecionar só met_vars e seus lags
features = [c for c in cruzamento_dados.columns if is_met_var(c)]

X = cruzamento_dados[features]
y = cruzamento_dados['casos']

In [ ]:
# SPLIT TEMPORAL (70% TREINO, 30% TESTE)
n = len(cruzamento_dados)
n_train = int(0.7 * n)

X_train = X.iloc[:n_train]
X_test  = X.iloc[n_train:]
y_train = y.iloc[:n_train]
y_test  = y.iloc[n_train:]

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
resultado = decision_tree.predict(X_test)
metrics.recall_score(y_test, resultado, average="micro")

In [ ]:
valores=pd.DataFrame(columns=['indice','porcentagem'])
# for i in range(1, 100):
random_forest = RandomForestClassifier(n_estimators=18)
random_forest.fit(X_train, y_train)
resultado = random_forest.predict(X_test)

resultado_porcentagem=metrics.recall_score(y_test, resultado, average="micro")
print(classification_report(y_test, resultado))

valores.loc[len(valores)] = [18, resultado_porcentagem]
print(18,metrics.recall_score(y_test, resultado, average="micro"))

In [ ]:
max_porcentagem_index = valores['porcentagem'].idxmax()

indice_at_max_porcentagem = valores.loc[max_porcentagem_index, 'indice']

print(f"The 'indice' value corresponding to the maximum 'porcentagem' is: {indice_at_max_porcentagem}, valor {valores['porcentagem'].max()}")


In [ ]:
importances = random_forest.feature_importances_
for name, score in zip(X.columns, importances):
    print(f"{name}: {score:.2f}")

In [ ]:
   import matplotlib.pyplot as plt
import pandas as pd

# Já temos importances e X.columns:
importances = random_forest.feature_importances_

# Criar DataFrame
df = pd.DataFrame({'Variável': X.columns, 'Importância': importances})

# Extrair nome base
df['Variável_base'] = df['Variável'].str.extract(r'(^[^\(_]+)')

# Agrupar e somar importâncias
df_grouped = df.groupby('Variável_base')['Importância'].sum().reset_index()

# Ordenar
df_grouped = df_grouped.sort_values(by='Importância', ascending=True)

# Plotar gráfico
plt.figure(figsize=(12, 8))
plt.barh(df_grouped['Variável_base'], df_grouped['Importância'], color='forestgreen')
plt.title('Importância das Variáveis (Random Forest) - Agrupada por 4 semanas', fontsize=24, weight='bold')
plt.xlabel('Importância', fontsize=18)
plt.ylabel('Variável', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.gca().set_facecolor('white')
plt.tight_layout()
plt.savefig("importancia_random_forest_banner.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
 cruzamento_dados

In [ ]:
# prompt: gere uma matriz de confusão

import matplotlib.pyplot as plt
# Gera a matriz de confusão
cm = confusion_matrix(y_test, resultado, labels=random_forest.classes_) # Use random_forest.classes_ if you are using Random Forest for classification

# Cria o gráfico com estilo profissional
fig, ax = plt.subplots(figsize=(10, 8))  # Tamanho maior para banner

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=random_forest.classes_) # Use random_forest.classes_

disp.plot(ax=ax, cmap='Blues', colorbar=False)

# Ajustes de estilo
plt.title("Matriz de Confusão\nClassificação do Nível da Dengue - Fortaleza (2015–2024)",
          fontsize=20, weight='bold', pad=20)
plt.xlabel("Previsão do Modelo", fontsize=16)
plt.ylabel("Valor Real", fontsize=16)

# Ajuste nos ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Fundo branco
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

plt.tight_layout()
plt.savefig("matriz_confusao_banner.png", dpi=300)
plt.show()


In [ ]:
# Gera a matriz de confusão
cm = confusion_matrix(y_test, resultado, labels=decision_tree.classes_)

# Cria o gráfico com os nomes das classes
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=decision_tree.classes_)

fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax, cmap='Blues', colorbar=False)

plt.title("Matriz de Confusão - Classificação de Nível da Dengue")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Gera a matriz de confusão
cm = confusion_matrix(y_test, resultado, labels=decision_tree.classes_)

# Cria o gráfico com estilo profissional
fig, ax = plt.subplots(figsize=(10, 8))  # Tamanho maior para banner

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=decision_tree.classes_)

disp.plot(ax=ax, cmap='Blues', colorbar=False)

# Ajustes de estilo
plt.title("Matriz de Confusão\nClassificação do Nível da Dengue - Fortaleza (2015–2024)",
          fontsize=20, weight='bold', pad=20)
plt.xlabel("Previsão do Modelo", fontsize=16)
plt.ylabel("Valor Real", fontsize=16)

# Ajuste nos ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Fundo branco
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

plt.tight_layout()
plt.show()
